### Processor
* VOC_duration : 3sec
* Wash_Out : 8min

### Sensors
+ TGS : TGS2600, TGS2602, TGS2603, TGS2610, TGS2620
+ MQ : MQ2, MQ3, MQ4, MQ5, MQ6, MQ7, MQ8, MQ9, MQ135

### VOC
1. C.albicans
2. C.glabrata
3. E.coli
4. K.pneumoniae
5. S.aureus
6. S.epidermidis

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pymysql
import matplotlib.pyplot as plt
# import config
import datetime, time
import os
from pathlib import Path
from tqdm import tqdm
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
plot_style = plt.style.available
# pd.options.display.float_format = '{:.5f}'.format
# pd.set_option('display.max_row', 500)
pd.set_option('display.max_rows', 500)

In [3]:
# plt.style.use('ggplot')
# plt.style.use('default')
# plt.style.use('fivethirtyeight')
plt.style.use('seaborn-paper')
# plt.style.use('seaborn-muted')

In [145]:
pd.set_option('display.expand_frame_repr', False)
np.random.seed(42)

In [9]:
PROJECT_DIR = Path('.')
DATA_DIR = PROJECT_DIR / 'data'

In [12]:
print(PROJECT_DIR, DATA_DIR)

. data


In [135]:
with pd.HDFStore(PROJECT_DIR/'microbe_data_01.h5') as store:
    print(store.info())
    df = store['microbe/raw_data']

<class 'pandas.io.pytables.HDFStore'>
File path: microbe_data_01.h5
/microbe/raw_data            frame        (shape->[7166968,20])


In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7166968 entries, ('C.albicans_sensor_1', 0, 0.0) to ('S.epidermidis_sensor_3', 120, 20.738000128)
Data columns (total 20 columns):
 #   Column     Dtype         
---  ------     -----         
 0   Time       datetime64[ns]
 1   TGS2600    float64       
 2   TGS2602    float64       
 3   TGS2603    float64       
 4   TGS2620    float64       
 5   TGS2610    float64       
 6   TGS822     float64       
 7   MQ2        float64       
 8   MQ3        float64       
 9   MQ4        float64       
 10  MQ5        float64       
 11  MQ6        float64       
 12  MQ7        float64       
 13  MQ8        float64       
 14  MQ9        float64       
 15  MQ135      float64       
 16  stamp      float64       
 17  VOC        int64         
 18  FILE_NAME  object        
 19  VALVE      object        
dtypes: datetime64[ns](1), float64(16), int64(1), object(2)
memory usage: 1.2+ GB


In [138]:
df_temp.index.get_level_values(0)

Index(['C.albicans_sensor_1', 'C.albicans_sensor_1', 'C.albicans_sensor_1',
       'C.albicans_sensor_1', 'C.albicans_sensor_1', 'C.albicans_sensor_1',
       'C.albicans_sensor_1', 'C.albicans_sensor_1', 'C.albicans_sensor_1',
       'C.albicans_sensor_1',
       ...
       'C.albicans_sensor_1', 'C.albicans_sensor_1', 'C.albicans_sensor_1',
       'C.albicans_sensor_1', 'C.albicans_sensor_1', 'C.albicans_sensor_1',
       'C.albicans_sensor_1', 'C.albicans_sensor_1', 'C.albicans_sensor_1',
       'C.albicans_sensor_1'],
      dtype='object', name='SAMPLE', length=2660)

In [139]:
sensors = ['TGS2600', 'TGS2602', 'TGS2603', 'TGS2620', 'TGS2610', 'TGS822', 
           'MQ2', 'MQ3', 'MQ4', 'MQ5', 'MQ6', 'MQ7', 'MQ8', 'MQ9', 'MQ135']

In [1]:
idx = pd.IndexSlice
samples = df.index.get_level_values('SAMPLE').unique().tolist()

for sample in samples[:]:
    df_temp = df.loc[idx[str(sample), :, :], :]
    cycles = df_temp.index.get_level_values('CYCLE').unique().tolist()

    for cycle in cycles[:]:
        df_temp_temp = df_temp.loc[idx[:, cycle:cycle, :], :]
              
        for sensor in sensors:
            df_temp_sensor = df_temp_temp[sensor].dropna()
#             plt.plot(df_temp_sensor.index.get_level_values(2), df_temp_sensor)
            if len(df_temp_sensor) == 0:
                continue
            df_temp_sensor_ratio = df_temp_sensor
#             df_temp_sensor_ratio = df_temp_sensor / df_temp_sensor[0] # 비율
            plt.plot(df_temp_sensor_ratio.index.get_level_values('TIME_RELATIVE'), df_temp_sensor_ratio)
#             plt.legend(sensors, loc='upper left', bbox_to_anchor=(1.02, 1))
            valve_on = df_temp_temp[df_temp_temp['VALVE'] == 'open'].index.get_level_values('TIME_RELATIVE')
            plt.axvspan(valve_on.min(), valve_on.max(), color="gray", alpha=0.1)
            
        title = str(sample)
        title = title + ', VOC: ' + str(df_temp_temp['VOC'].iloc[0])
        title = title + ', Cycle: ' + str(cycle) + '/' + str(len(cycles))
        plt.title(title, fontsize=13, loc="left")
#         plt.legend(loc='upper left', bbox_to_anchor=(1.02, 1))
        plt.show()


NameError: name 'pd' is not defined